In [4]:
!pip install bs4
!pip install requests

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [5]:
import requests

In [56]:
topics_url = ('https://www.imdb.com/search/title/?groups=top_100&sort=user_rating,desc')


In [57]:
response = requests.get(topics_url)

In [58]:
page_content = response.text

In [91]:
page_content[:500]

'\n\n<!DOCTYPE html>\n<html\n    xmlns:og="http://ogp.me/ns#"\n    xmlns:fb="http://www.facebook.com/2008/fbml">\n    <head>\n         \n\n        <meta charset="utf-8">\n\n\n\n\n        <script type="text/javascript">var IMDbTimer={starttime: new Date().getTime(),pt:\'java\'};</script>\n\n<script>\n    if (typeof uet == \'function\') {\n      uet("bb", "LoadTitle", {wb: 1});\n    }\n</script>\n  <script>(function(t){ (t.events = t.events || {})["csm_head_pre_title"] = new Date().getTime(); })(IMDbTimer);</script>\n      '

In [7]:
from bs4 import BeautifulSoup

In [9]:
import pandas as pd

In [60]:
doc = BeautifulSoup(response.text, 'html.parser')



In [61]:
doc.find('title')



<title>IMDb "Top 100"
(Sorted by IMDb Rating Descending) - IMDb</title>

In [62]:
def get_topics_page():
    # TODO - add comments
    topic_url = 'https://www.imdb.com/search/title/?groups=top_100&sort=user_rating,desc'
    response=requests.get(topic_url)
    # check successfull response
    if response.status_code != 200:
        raise Exception(f'Failed to load page {topic_url}')
    # Parse using BeautifulSoup
    doc = BeautifulSoup(response.text, 'html.parser')
    return doc

In [63]:
doc=get_topics_page()

In [64]:
def get_movie_titles(doc):
    
    selection_class="lister-item-header"
    movie_title_tags=doc.find_all('h3',{'class':selection_class})
    movie_titles=[]

    for tag in movie_title_tags:
        title = tag.find('a').text
        movie_titles.append(title)
        
        
    return movie_titles

In [65]:
titles = get_movie_titles(doc)

In [92]:
titles[:5]

['The Shawshank Redemption',
 'The Godfather',
 'The Dark Knight',
 'The Lord of the Rings: The Return of the King',
 "Schindler's List"]

In [67]:
def get_movie_year(doc):
    year_selector = "lister-item-year text-muted unbold"           
    movie_year_tags=doc.find_all('span',{'class':year_selector})
    movie_year_tagss=[]
    for tag in movie_year_tags:
        movie_year_tagss.append(tag.get_text().strip()[1:5])
    return movie_year_tagss

In [68]:
years = get_movie_year(doc)

In [93]:
years[:5]

['1994', '1972', '2008', '2003', '1993']

In [70]:
def get_movie_rating(doc):
    rating_selector="inline-block ratings-imdb-rating"            
    movie_rating_tags=doc.find_all('div',{'class':rating_selector})
    movie_rating_tagss=[]
    for tag in movie_rating_tags:
        movie_rating_tagss.append(tag.get_text().strip())
    return movie_rating_tagss

In [71]:
ratings = get_movie_rating(doc)

In [94]:
ratings[:5]

['9.3', '9.2', '9.0', '9.0', '9.0']

In [105]:
def all_pages():
# Let's we create a dictionary to store data of all movies
    movies_dict={
        'title':[],    
        'rating':[],
        'year':[],
       
    }
  # We have to scrap more than one page so we want urls of all pages with the help of loop we can get all urls
    for i in range(1,2000,100):
       
        try:
            url = 'https://www.imdb.com/search/title/?groups=top_100&sort=user_rating,desc'
            response = requests.get(url)
        except:
            break
        
        if response.status_code != 200:
            break
           
    # Parse using BeautifulSoup
        doc = BeautifulSoup(response.text, 'html.parser')
        titles = get_movie_titles(doc)
       
       
        ratings = get_movie_rating(doc)
       
        years = get_movie_year(doc)
      
    
        
    # We are adding every movie data to dictionary
        for i in range(len(titles)):
            movies_dict['title'].append(titles[i])
         
         
            movies_dict['rating'].append(ratings[i])
            movies_dict['year'].append(years[i])
           
        
    return pd.DataFrame(movies_dict)

In [101]:
movies_dict={
    'title':titles,
    'rating':ratings,
    'year':years,
    
}

In [102]:
movies_dict

{'title': ['The Shawshank Redemption',
  'The Godfather',
  'The Dark Knight',
  'The Lord of the Rings: The Return of the King',
  "Schindler's List",
  'The Godfather Part II',
  '12 Angry Men',
  'Pulp Fiction',
  'Inception',
  'The Lord of the Rings: The Two Towers',
  'Fight Club',
  'The Lord of the Rings: The Fellowship of the Ring',
  'Forrest Gump',
  'Il buono, il brutto, il cattivo',
  'The Matrix',
  'Goodfellas',
  'The Empire Strikes Back',
  "One Flew Over the Cuckoo's Nest",
  'Interstellar',
  'Cidade de Deus',
  'Sen to Chihiro no kamikakushi',
  'Saving Private Ryan',
  'The Green Mile',
  'La vita è bella',
  'Se7en',
  'Terminator 2: Judgment Day',
  'The Silence of the Lambs',
  'Star Wars',
  'Seppuku',
  'Shichinin no samurai',
  "It's a Wonderful Life",
  'Gisaengchung',
  'Whiplash',
  'The Intouchables',
  'The Prestige',
  'The Departed',
  'The Pianist',
  'Gladiator',
  'American History X',
  'The Usual Suspects',
  'Léon',
  'The Lion King',
  'Nuovo Ci

In [103]:
df = pd.DataFrame(movies_dict)

In [104]:
df.head()

,title,rating,year
0,The Shawshank Redemption,9.3,1994
1,The Godfather,9.2,1972
2,The Dark Knight,9.0,2008
3,The Lord of the Rings: The Return of the King,9.0,2003
4,Schindler's List,9.0,1993
